In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [2]:
poker = pd.read_csv("poker.csv")
poker.columns.values
poker = poker.head(200000)

In [3]:
i = 100
j = 0
features = ['S1', ' C1', ' S2', ' C2', ' S3', ' C3', ' S4', ' C4', ' S5', ' C5']
for index, m in poker.iterrows():
    if index % 20 == 0:
        poker.at[index,features[j]] = i + 1
        j += 3
        i+=10
        if j >= 9:
            j = 0

In [5]:
i = 0
j = 0
for index, m in poker.iterrows():
    if index % 20 == 0:
        if index < 199994:
            poker.at[index+5,:] = poker.loc[j,:]
        else:
            poker.at[index+1,:] = poker.loc[j,:]
        j += 120
        if j >= 199994:
            j = 0

In [7]:
poker.to_csv('pokerNoise.csv', sep=',')

In [8]:
poker

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,Class
0,101,12,3,2,3,11,4,5,2,5,1
1,1,9,4,6,1,4,3,2,3,9,1
2,1,4,3,13,2,13,2,1,3,6,1
3,3,10,2,7,1,2,2,11,4,9,0
4,1,3,4,5,3,4,1,12,4,6,0
5,101,12,3,2,3,11,4,5,2,5,1
6,3,2,4,9,3,7,4,3,4,5,0
7,4,4,3,13,1,8,3,9,3,10,0
8,1,9,3,8,4,4,1,7,3,5,0
9,4,7,3,12,1,13,1,9,2,6,0


In [9]:
data = poker.iloc[:,0:8]
labels = poker.iloc[:,8].apply(str)

In [12]:
x = data.values.astype(np.float32) #returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_scaled = pd.DataFrame(x_scaled)
poker = pd.concat([df_scaled, labels], axis=1)

In [13]:
kf = KFold(n_splits=2, shuffle=True) #5 fores me 2 folds

### Euclidean and k tuning on 10% noise dataset

In [14]:
acc = []
mean_accuracy_model_euclidean = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=1, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.30      0.30      0.30     24941
          3       0.29      0.29      0.29     25043
          2       0.29      0.29      0.29     24984
          4       0.29      0.29      0.29     25032

avg / total       0.29      0.29      0.29    100000

accuracy:  0.29367
             precision    recall  f1-score   support

          1       0.30      0.30      0.30     25148
          3       0.29      0.29      0.29     24813
          2       0.29      0.29      0.29     24862
          4       0.30      0.30      0.30     25177

avg / total       0.29      0.29      0.29    100000

accuracy:  0.29461
             precision    recall  f1-score   support

          1       0.30      0.30      0.30     25147
          3       0.29      0.29      0.29     24706
          2       0.29      0.29      0.29     24997
          4       0.30      0.30      0.30     25150

avg / total       0.29      0.29      0.29    100000


In [15]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=5, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.28      0.37      0.32     25016
          3       0.28      0.30      0.29     25013
          2       0.28      0.26      0.27     24802
          4       0.30      0.20      0.24     25169

avg / total       0.29      0.28      0.28    100000

accuracy:  0.28373
             precision    recall  f1-score   support

          1       0.28      0.38      0.32     25073
          3       0.28      0.30      0.29     24843
          2       0.29      0.25      0.27     25044
          4       0.29      0.20      0.24     25040

avg / total       0.28      0.28      0.28    100000

accuracy:  0.28326
             precision    recall  f1-score   support

          1       0.28      0.38      0.32     24891
          3       0.28      0.30      0.29     25009
          2       0.29      0.25      0.27     24905
          4       0.29      0.20      0.24     25195

avg / total       0.28      0.28      0.28    100000


In [16]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=10, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.26      0.35      0.30     24972
          3       0.26      0.27      0.27     24937
          2       0.27      0.24      0.25     24952
          4       0.27      0.20      0.23     25139

avg / total       0.27      0.27      0.26    100000

accuracy:  0.26617
             precision    recall  f1-score   support

          1       0.27      0.36      0.31     25117
          3       0.27      0.28      0.27     24919
          2       0.27      0.23      0.25     24894
          4       0.28      0.21      0.24     25070

avg / total       0.27      0.27      0.27    100000

accuracy:  0.27069
             precision    recall  f1-score   support

          1       0.27      0.35      0.30     25174
          3       0.26      0.27      0.27     24899
          2       0.27      0.23      0.25     24858
          4       0.27      0.21      0.24     25069

avg / total       0.27      0.27      0.26    100000


In [17]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=15, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.27      0.34      0.30     25052
          3       0.26      0.27      0.27     24993
          2       0.26      0.23      0.25     24880
          4       0.27      0.22      0.24     25075

avg / total       0.27      0.27      0.26    100000

accuracy:  0.26551
             precision    recall  f1-score   support

          1       0.26      0.34      0.29     25037
          3       0.26      0.27      0.26     24863
          2       0.26      0.23      0.25     24966
          4       0.27      0.22      0.24     25134

avg / total       0.26      0.26      0.26    100000

accuracy:  0.26311
             precision    recall  f1-score   support

          1       0.26      0.34      0.29     25003
          3       0.26      0.26      0.26     25066
          2       0.26      0.24      0.25     24676
          4       0.27      0.22      0.25     25255

avg / total       0.27      0.26      0.26    100000


In [18]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=20, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.26      0.32      0.29     25117
          3       0.26      0.27      0.26     24863
          2       0.27      0.24      0.25     24963
          4       0.27      0.23      0.25     25057

avg / total       0.27      0.26      0.26    100000

accuracy:  0.26478
             precision    recall  f1-score   support

          1       0.26      0.33      0.29     24972
          3       0.26      0.26      0.26     24993
          2       0.27      0.24      0.25     24883
          4       0.28      0.23      0.25     25152

avg / total       0.27      0.27      0.26    100000

accuracy:  0.26545
             precision    recall  f1-score   support

          1       0.26      0.32      0.29     25002
          3       0.26      0.26      0.26     25086
          2       0.26      0.24      0.25     24809
          4       0.27      0.23      0.25     25103

avg / total       0.26      0.26      0.26    100000


In [19]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=30, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.26      0.32      0.28     25011
          3       0.26      0.25      0.26     25094
          2       0.26      0.24      0.25     24920
          4       0.27      0.24      0.25     24975

avg / total       0.26      0.26      0.26    100000

accuracy:  0.26007
             precision    recall  f1-score   support

          1       0.26      0.32      0.29     25078
          3       0.26      0.26      0.26     24762
          2       0.26      0.24      0.25     24926
          4       0.27      0.23      0.25     25234

avg / total       0.26      0.26      0.26    100000

accuracy:  0.26186
             precision    recall  f1-score   support

          1       0.26      0.31      0.28     25053
          3       0.26      0.26      0.26     24912
          2       0.27      0.24      0.25     24977
          4       0.26      0.23      0.25     25058

avg / total       0.26      0.26      0.26    100000


In [20]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=50, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.26      0.30      0.28     25255
          3       0.26      0.24      0.25     25041
          2       0.26      0.25      0.25     24735
          4       0.26      0.25      0.25     24969

avg / total       0.26      0.26      0.26    100000

accuracy:  0.25794
             precision    recall  f1-score   support

          1       0.26      0.32      0.28     24834
          3       0.26      0.25      0.25     24815
          2       0.26      0.23      0.24     25111
          4       0.26      0.23      0.24     25240

avg / total       0.26      0.26      0.26    100000

accuracy:  0.25748
             precision    recall  f1-score   support

          1       0.26      0.31      0.28     25023
          3       0.26      0.25      0.26     24928
          2       0.26      0.23      0.24     25057
          4       0.26      0.25      0.25     24992

avg / total       0.26      0.26      0.26    100000


In [21]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=100, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.26      0.31      0.28     24935
          3       0.26      0.24      0.25     24942
          2       0.25      0.23      0.24     24957
          4       0.26      0.25      0.25     25166

avg / total       0.26      0.26      0.26    100000

accuracy:  0.25657
             precision    recall  f1-score   support

          1       0.26      0.29      0.28     25154
          3       0.25      0.24      0.24     24914
          2       0.25      0.23      0.24     24889
          4       0.25      0.27      0.26     25043

avg / total       0.26      0.26      0.25    100000

accuracy:  0.25545
             precision    recall  f1-score   support

          1       0.25      0.30      0.28     25059
          3       0.25      0.23      0.24     24834
          2       0.25      0.23      0.24     24880
          4       0.26      0.25      0.26     25227

avg / total       0.25      0.25      0.25    100000


In [22]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(poker):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=150, algorithm = 'auto', metric = 'euclidean')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)

mean_accuracy_model1 =  sum(acc)/10    
mean_accuracy_model_euclidean.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model1

             precision    recall  f1-score   support

          1       0.25      0.31      0.28     24955
          3       0.26      0.24      0.25     24925
          2       0.25      0.21      0.23     24991
          4       0.26      0.26      0.26     25129

avg / total       0.25      0.25      0.25    100000

accuracy:  0.25447
             precision    recall  f1-score   support

          1       0.25      0.29      0.27     25134
          3       0.26      0.22      0.24     24931
          2       0.25      0.23      0.24     24855
          4       0.25      0.27      0.26     25080

avg / total       0.25      0.25      0.25    100000

accuracy:  0.25377
             precision    recall  f1-score   support

          1       0.26      0.32      0.28     24931
          3       0.26      0.24      0.25     24895
          2       0.25      0.22      0.24     24915
          4       0.25      0.25      0.25     25259

avg / total       0.26      0.26      0.25    100000


In [23]:
print mean_accuracy_model_euclidean
k = [1, 5, 10, 15, 20, 30, 50, 100, 150]
fig = plt.figure()
ax = plt.axes()
ax.plot(k, mean_accuracy_model_euclidean)
plt.show()

[0.294618, 0.283019, 0.267747, 0.26432300000000003, 0.264174, 0.2616820000000001, 0.25806999999999997, 0.255506, 0.2548210000000001]


NameError: name 'plt' is not defined

### Minkowski and k tuning on 10% dataset

In [ ]:
acc = []
mean_accuracy_model_minkowski = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=1, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=5, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=10, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=15, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=20, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=30, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=50, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=100, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
acc = []
for i in range(5):
    for train_index, test_index in kf.split(nurseryNum):
        X_train, X_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        all_labels = pd.concat([y_train, y_test], axis=0)
        knn = KNeighborsClassifier(n_neighbors=150, algorithm = 'auto', metric = 'minkowski')
        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        if len(set(predicted)) == len(set(y_test)):
            unique_labels = set(predicted)
            print unique_labels
        elif len(set(predicted)) < len(set(y_test)):
            unique_labels = set(y_test)
        else :
            unique_labels = set(predicted)
        print classification_report(y_test, predicted, target_names=unique_labels)
        acc.append(accuracy_score(y_test, predicted))
        print "accuracy: ", accuracy_score(y_test, predicted)
        
mean_accuracy_model10 = sum(acc)/10  
mean_accuracy_model_minkowski.append(sum(acc)/10)
print "mean accuracy", mean_accuracy_model10

In [ ]:
print mean_accuracy_model_minkowski
k = [1, 5, 10, 15, 20, 30, 50, 100, 150]
fig = plt.figure()
ax = plt.axes()
ax.plot(k, mean_accuracy_model_minkowski)
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.plot(k, mean_accuracy_model_minkowski)
ax.plot(k, mean_accuracy_model_euclidean)
plt.show()